In [69]:
#@title SPRAWDŹ SWOJE POLITYCZNE PREFERENCJE

ID = "LechPoznan" #@param ["", ""] {allow-input: true}


In [78]:
import tweepy
import pandas as pd
import time

# Credentials

consumer_key = "5MzB6K9I3B0hVF0BOs2UjybSz"
consumer_secret = "hiPjP10G7bTuomHflC1vCWzwcdsAB444WkE0MkSZwGGfALoxBS"
access_token = "1437020925040513029-s6Dqtv9lPyufnlKtzm9tF3BQEQ8FCA"
access_token_secret = "D9SUsW12b1ya9LuWPNNmGmU0NhKIJzoeFfHsIy4xgdJ7x"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

tweets = []

def username_tweets_to_csv(username,count):
    try:      
        # Creation of query method using parameters
        #tweets = tweepy.Cursor(api.user_timeline,id=username).items(count)
        tweets = tweepy.Cursor(api.user_timeline,id=username, since='01-08-2021').items(count)

        # Pulling information from tweets iterable object
        tweets_list = [[tweet.user.screen_name, tweet.created_at, tweet.id, tweet.text] for tweet in tweets]

        # Creation of dataframe from tweets list
        # Add or remove columns as you remove tweet information
        tweets_df = pd.DataFrame(tweets_list,columns=['Author', 'Datetime', 'Tweet Id', 'Text'])
        

        # Converting dataframe to CSV 
        #tweets_df.to_csv('{}.csv'.format(username), sep=',', index = False)
        return tweets_df

    except BaseException as e:
          print('failed on_status,',str(e))
          time.sleep(3)


count = 1000
data = username_tweets_to_csv(ID, count)

In [79]:
data

,Author,Datetime,Tweet Id,Text
0,LechPoznan,2021-09-19 09:35:49,1439523612906729475,Za 9️⃣ dni znowu widzimy się przy Bułgarskiej....
1,LechPoznan,2021-09-18 19:32:01,1439311266640809991,1⃣8⃣3⃣2⃣ dni czekaliśmy na ligowego gola zawod...
2,LechPoznan,2021-09-18 17:53:03,1439286361362735107,𝚆𝚎𝚎𝚔𝚎𝚗𝚍 𝟼/𝟼\n\n☑️ Lech Poznań - Wisła Kraków 5...
3,LechPoznan,2021-09-18 15:13:27,1439246193725431808,👏🏼 https://t.co/QOl9Q8BHlA
4,LechPoznan,2021-09-18 13:59:42,1439227634802647047,Braaaaawo! 🤝 https://t.co/tgdY0EkPkt
...,...,...,...,...
995,LechPoznan,2021-04-17 16:32:12,1383458280542916619,⏱️ 46' Początek drugiej połowy meczu. \n\nRela...
996,LechPoznan,2021-04-17 16:25:53,1383456691023347714,⏱️ 45'+1 ⏸️ Pierwsza połowa meczu dla gospodar...
997,LechPoznan,2021-04-17 15:59:38,1383450088312164360,⏱️ 29' Drugi gol dla gospodarzy. Piątkowski. \...
998,LechPoznan,2021-04-17 15:42:43,1383445829604704259,⏱️ 10' ⚽️ Gospodarze wychodzą na prowadzenie. ...


In [53]:
pip install dash pandas

     |████████████████████████████████| 7.3 MB 28.9 MB/s 
     |████████████████████████████████| 23.9 MB 14 kB/s 
     |████████████████████████████████| 357 kB 51.8 MB/s 
  Created wheel for dash-core-components: filename=dash_core_components-2.0.0-py3-none-any.whl size=3821 sha256=83b1d8d8763d024a61c8d06f0eec1cf8bb3f5cdd68ebde862d11938dab78d0ef
  Stored in directory: /root/.cache/pip/wheels/00/f9/c7/1a6437d794ed753ea9bc9079e761d4fc803a1f1f5d3697b9ec
  Created wheel for dash-html-components: filename=dash_html_components-2.0.0-py3-none-any.whl size=4089 sha256=bf9af2601f7d1a7da7690cbce274b48cccbb649d7849e140d0e5ce7c883be62f
  Stored in directory: /root/.cache/pip/wheels/ec/6b/81/05aceabd8b27f724e2c96784016287cc1bfbc349ebfda451de
  Created wheel for dash-table: filename=dash_table-5.0.0-py3-none-any.whl size=3911 sha256=cf1c2183b53f3a39760206d6ebb841c1c066bef4ffe1b180e805ac9deb299d71
  Stored in directory: /root/.cache/pip/wheels/85/5d/4e/7c276b57992951dbe770bf5caad6448d0539c510663aef

In [61]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd

external_stylesheets = [
    'https://codepen.io/chriddyp/pen/bWLwgP.css'
]

# Create Dash app
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
# Expose Flask instance
server = app.server

# Alternative version, using flask server directly (add import flask in imports)
# server = flask.Flask(__name__)
# app = dash.Dash(__name__, external_stylesheets=external_stylesheets, server=server)

# Assume you have a "long-form" data frame
# see https://plotly.com/python/px-arguments/ for more options
df = pd.DataFrame({
    "Fruit": ["Apples", "Oranges", "Bananas", "Apples", "Oranges", "Bananas"],
    "Amount": [4, 1, 2, 2, 4, 5],
    "City": ["SF", "SF", "SF", "Montreal", "Montreal", "Montreal"]
})

fig = px.bar(
    df, 
    x="Fruit", y="Amount", color="City", barmode="group"
)

app.layout = html.Div(children=[
    html.H1(children='Hello Dash'),

    html.Div(children='''
        Dash: A web application framework for Python. Customized right here!
    '''),
    dcc.Graph(
        id='example-graph',
        figure=fig
    )
])

# Only for running on development mode
if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
